In [30]:
import pandas as pd, duckdb
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os
import polars as pl

In [ ]:
df_ls_firms = pd.read_stata('/kellogg/proj/lgg3230/UnionSpill/Data/CBA_RAIS_firm_level/lagos_sample_sep24.dta', convert_categoricals=False ) # Load lagos sample data
# df_workers_all = pd.read_stata('/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/worker_estab_all_years.dta', convert_categoricals=False) # Load worker-establishment data in chunks
# len(df_ls_firms)

174110

In [3]:
table1_sample = pa.Table.from_pandas(df1_sample)
pq.write_table(table1_sample, '/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/worker_estab_all_years.parquet')

In [4]:
table = pa.Table.from_pandas(df0)
pq.write_table(table, '/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/lagos_sample_year.parquet')

In [5]:
df = pl.read_parquet("/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/worker_estab_all_years.parquet")

In [6]:
con = duckdb.connect()
con.execute("pragma threads=5")  # Set number of threads for DuckDB

In [7]:
con.register("df0",df0)

In [8]:
con.register("df",df)

* see if merged dataset does not have missing values in the incomes from firms.
* then:
    - define swit


In [9]:
con.execute(f"""
            copy(
            select df.*
            from df
            left join df0 using (cnpj_year)
            )
            to '/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/worker_estab_lagos_sample.parquet'
            """)

In [10]:
df_merged = pd.merge(df1_sample, df0, on='cnpj_year', how='left',suffixes=('_worker','_estab'))

In [11]:
df_merged.head()

,year,year_str_worker,PIS,identificad_worker,municipio,tipoadm,causadesli,mesdesli,ocup2002,grinstrucao,...,r_remmedr_h,r_remdezr_h,lr_remdezr,r_remdezr,dtnascimento_stata,modeind,modemun,cnpj_year,identificad_estab,year_str_estab
0,2011,2011,14061872725,00000000000070,421660,0,0,0,422110,7,...,6.758034,1.911361,6.782274,882.072388,NaT,86909,421660,000000000000702011,NaN,NaN
1,2009,2009,13286190429,00000000000272,130260,0,0,0,413210,9,...,28.962761,3.622895,8.493809,4884.432617,1981-11-18,64221,130260,000000000002722009,NaN,NaN
2,2009,2009,18000161295,00000000000272,130260,0,0,0,413225,9,...,84.034782,4.429561,9.588156,14590.938477,1966-11-08,64221,130260,000000000002722009,NaN,NaN
3,2013,2013,10711405597,00000000000272,130260,0,0,0,413225,5,...,95.389946,4.603624,9.474537,13023.844727,NaT,64221,130260,000000000002722013,NaN,NaN
4,2011,2011,19018604987,00000000000272,130260,4,0,0,413225,9,...,136.087906,4.625635,9.784230,17751.582031,NaT,64221,130260,000000000002722011,NaN,NaN


In [12]:
df_lagos = pl.read_parquet("/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/worker_estab_lagos_sample.parquet")

In [17]:
df_lagos_pd = df_lagos.to_pandas()

In [13]:
df_lagos.head()

year,year_str,PIS,identificad,municipio,tipoadm,causadesli,mesdesli,ocup2002,grinstrucao,genero,dtnascimento,idade,nacionalidad,portdefic,tpdefic,raca_cor,remdezembro,remmedia,remdezr,remmedr,tempempr,tiposal,salcontr,ultrem,horascontr,clascnae20,dtadmissao_stata,lr_remmedr,r_remmedr,r_remmedr_h,r_remdezr_h,lr_remdezr,r_remdezr,dtnascimento_stata,modeind,modemun,cnpj_year
i16,str,str,str,i32,i8,i8,i8,i32,i8,i8,str,i16,i8,i8,i8,i8,f64,f64,f64,f64,f64,i8,f64,f64,i8,str,datetime[ns],f32,f32,f32,f32,f32,f32,datetime[ns],str,i32,str
2014,"""2014""","""12943075606""","""00000000471208""",330455,4,0,0,413225,9,0,"""28091976""",38,10,0,0,2,3.26,3.89,2362.89,2821.34,2.5,1,2362.89,2362.89,30,"""64221""",2014-10-13 00:00:00,8.046377,3122.462402,23.937922,2.998137,7.869051,2615.082031,1976-09-28 00:00:00,"""64221""",330455,"""000000004712082014"""
2014,"""2014""","""18000618899""","""00000000471801""",290682,0,0,0,413225,7,1,"""23041977""",37,10,0,0,8,8.31,8.48,6020.25,6145.0,21.8,1,2433.77,6020.25,40,"""64221""",2013-03-04 00:00:00,8.824804,6800.857422,39.103367,3.645699,8.804294,6662.792969,1977-04-23 00:00:00,"""64221""",290682,"""000000004718012014"""
2014,"""2014""","""10726883260""","""00000000471801""",290682,0,0,0,413210,7,1,"""12021957""",57,10,0,0,8,6.58,6.04,4764.25,4382.0,33.8,1,2838.99,4764.25,30,"""64221""",2012-03-05 00:00:00,8.48667,4849.691895,37.179485,3.699392,8.570306,5272.739746,1957-02-12 00:00:00,"""64221""",290682,"""000000004718012014"""
2014,"""2014""","""13081011603""","""00000000471801""",290682,2,0,0,413225,9,1,"""25041989""",25,10,0,0,8,3.67,3.35,2664.17,2432.94,10.6,1,2294.06,2664.17,30,"""64221""",2014-02-10 00:00:00,7.898266,2692.608398,20.642506,3.118145,7.989058,2948.517578,1989-04-25 00:00:00,"""64221""",290682,"""000000004718012014"""
2009,"""2009""","""18000315276""","""00000000472441""",530010,4,0,0,413225,9,1,"""31031971""",38,10,1,0,2,25.04,25.34,11648.17,11664.06,11.8,1,2737.66,25.04,40,"""64221""",2009-01-05 00:00:00,9.762367,17367.703125,99.860291,4.602409,9.761004,17344.042969,1971-03-31 00:00:00,"""64221""",530010,"""000000004724412009"""


In [14]:
df_ls_sep24 = pd.read_stata('/kellogg/proj/lgg3230/UnionSpill/Data/CBA_RAIS_firm_level/lagos_sample_sep24.dta', convert_categoricals=False )
df_ls_sep24.head()

,identificad,year,l_firm_emp,lr_remdezr,lr_remmedr,l_firm_emp_2009,l_firm_emp_2009_5,lr_remdezr_2009,lr_remmedr_2009,totalout,...,totaltreat_pw_n_tf_n_pw_h,avg_ftreat_pf_n_am_tn_pre,avg_ftreat_pf_n_am_lfe,avg_ftreat_pf_n_am_tf_n_pw,avg_ftreat_pf_n_tn_pre_m,avg_ftreat_pf_n_tn_pre_h,avg_ftreat_pf_n_lfe_m,avg_ftreat_pf_n_lfe_h,avg_ftreat_pf_n_tf_n_pw_m,avg_ftreat_pf_n_tf_n_pw_h
0,00001961000110,2013.0,2.944439,6.935752,6.921747,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,00001961000110,2008.0,2.564949,6.480567,6.475050,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,00001961000110,2014.0,2.197225,7.100378,6.961579,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,00001961000110,2011.0,2.484907,6.827140,6.758582,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,00001961000110,2009.0,2.484907,6.891086,6.694481,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [15]:
len(df_ls_sep24[df_ls_sep24['in_balanced_panel'] == 1])

162760

In [22]:
con.register("df_lagos",df_lagos)

In [15]:
df_ls_sep24["cnpj_str"] = df_ls_sep24["identificad"].astype(str)
df_ls_sep24["year_str"] = df_ls_sep24["year"].astype(str)
df_ls_sep24["cnpj_year"] = df_ls_sep24["cnpj_str"] + df_ls_sep24["year_str"]
df_ls_sep24.head()

/tmp/ipykernel_2487687/377722729.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ls_sep24["cnpj_str"] = df_ls_sep24["identificad"].astype(str)
/tmp/ipykernel_2487687/377722729.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ls_sep24["year_str"] = df_ls_sep24["year"].astype(str)
/tmp/ipykernel_2487687/377722729.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(ax

,identificad,year,l_firm_emp,lr_remdezr,lr_remmedr,l_firm_emp_2009,l_firm_emp_2009_5,lr_remdezr_2009,lr_remmedr_2009,totalout,...,avg_ftreat_pf_n_am_tf_n_pw,avg_ftreat_pf_n_tn_pre_m,avg_ftreat_pf_n_tn_pre_h,avg_ftreat_pf_n_lfe_m,avg_ftreat_pf_n_lfe_h,avg_ftreat_pf_n_tf_n_pw_m,avg_ftreat_pf_n_tf_n_pw_h,cnpj_str,year_str,cnpj_year
0,00001961000110,2013.0,2.944439,6.935752,6.921747,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,00001961000110,2013.0,000019610001102013.0
1,00001961000110,2008.0,2.564949,6.480567,6.475050,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,00001961000110,2008.0,000019610001102008.0
2,00001961000110,2014.0,2.197225,7.100378,6.961579,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,00001961000110,2014.0,000019610001102014.0
3,00001961000110,2011.0,2.484907,6.827140,6.758582,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,00001961000110,2011.0,000019610001102011.0
4,00001961000110,2009.0,2.484907,6.891086,6.694481,2.484907,NaN,6.891086,6.694481,2.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,00001961000110,2009.0,000019610001102009.0


In [23]:
con.register("df_ls_sep24",df_ls_sep24)

In [28]:
# List of worker columns (with suffixes)
worker_cols = [f'w."{c}" AS "{c}_worker"' for c in df_lagos.columns if c != "cnpj_year"]

# Keep all firm columns (no suffix)
firm_cols = [f'f."{c}"' for c in df_ls_sep24.columns]

select_cols = ", ".join(worker_cols + firm_cols)

query = f"""
COPY (
    SELECT {select_cols}
    FROM df_lagos AS w
    LEFT JOIN df_ls_sep24 AS f
    USING (cnpj_year)
) TO '/kellogg/proj/lgg3230/UnionSpill/Data/CBA_RAIS_firm_level/worker_estab_lagos_sample_sep24.parquet'
  (FORMAT PARQUET, COMPRESSION ZSTD);
"""

con.execute(query)

In [29]:
df3 = pl.read_parquet("/kellogg/proj/lgg3230/UnionSpill/Data/CBA_RAIS_firm_level/worker_estab_lagos_sample_sep24.parquet")
len(df3)

1000000

In [20]:
df_merged_2 = pd.merge(df_lagos_pd, df_ls_sep24, on='cnpj_year', how='inner',suffixes=('_worker','_estab'))

In [21]:
df_merged_2.head()

,year_worker,year_str_worker,PIS,identificad_worker,municipio_worker,tipoadm,causadesli,mesdesli,ocup2002,grinstrucao,...,avg_ftreat_pf_n_am_lfe,avg_ftreat_pf_n_am_tf_n_pw,avg_ftreat_pf_n_tn_pre_m,avg_ftreat_pf_n_tn_pre_h,avg_ftreat_pf_n_lfe_m,avg_ftreat_pf_n_lfe_h,avg_ftreat_pf_n_tf_n_pw_m,avg_ftreat_pf_n_tf_n_pw_h,cnpj_str,year_str_estab


In [35]:
df_test = pd.read_parquet("/kellogg/proj/lgg3230/UnionSpill/Data/CBA_RAIS_firm_level/worker_estab_lagos_sample_sep24.parquet")